# Local Deployment

This example steps through how to deploy a local version of DTBase for development purposes. These instructions are a simplication of the developer docs and more details can be found there. 

There are two options for deploying DTBase locally:

1. Docker Compose 
2. Command Line 

Although docker compose is easier, it is faster to develop via deployment from the command line. This is because the docker images have to be rebuilt everytime a change is made to the code. The rest of this example will walk through the local deployment of DTBase via the command line. 

## Running DTBase Locally

### Fork Repository and Install DTBase

- Fork the DTBase repository: https://github.com/alan-turing-institute/DTBase and clone a copy to your local machine.  
- Navigate into the repository on the command line and install the dtbase package and dependencies

`pip install -e .[dev]`

### Set Environment Variables

Located in the .secrets folder, rename `dtenv_template.sh` to `dtenv_localdb.sh` (or any other name) and populate the variables. There is more information of what these values mean in the developer docs. The below is the minimal example needed to run dtbase locally. 

In [ ]:
export DT_SQL_TESTUSER="postgres"
export DT_SQL_TESTPASS="password"
export DT_SQL_TESTHOST="localhost"
export DT_SQL_TESTPORT="5432"
export DT_SQL_TESTDBNAME="test_db"

export DT_SQL_USER="postgres"
export DT_SQL_PASS="database_password"
export DT_SQL_HOST="localhost"
export DT_SQL_PORT="5432"
export DT_SQL_DBNAME="dt_dev"

export DT_BACKEND_URL="http://localhost:5000"
export DT_DEFAULT_USER_PASS="user_password"
export DT_FRONT_SECRET_KEY="frontend_password"
export DT_JWT_SECRET_KEY="jwt_password"

### Run a local database

Follow the instructions in the developer docs under the heading "Running a Local Database"

### Run Tests 

This is a good time to run the tests: 

`python -m pytest`

### Run the backend locally 

Follow instructions in developer docs under the heading "Running the backend API Locally"

### Run the frontend locally

Follow instructions in developer docs under the heading "Running the frontend locally"

## Testing the local deployment

There should now be three applications running: 

1. Docker Container running a Postgresql Database
2. Backend FastAPI app - `localhost:5000`
3. Frontend Flask app - `localhost:8000`

To test this out, we can navigate to the corresponding addresses. 

### Add a User

A good first test to check if the backend is successfully talking to the database is to add a user. This could can done either by the frontend or by sending a request to the backend programmatically. 

#### Frontend

![](images/dtbase_frontend.png)


#### Programatically

In [1]:
from dtbase.core.utils import auth_backend_call, login
import os

Load environment variables from environment file. Equivalent of sourcing the file in bash

In [2]:
env_vars = !cat ../.secrets/dtenv_localdb.sh
env_vars = [x for x in env_vars if x.startswith('export')]

for env_var in env_vars:
    key, value = env_var.split(' ')[1].split('=')
    # remove string quotes from value
    value = value.strip('\"')
    os.environ[key] = value

Login to the backend

In [3]:
username = 'default_user@localhost'
password = os.getenv("DT_DEFAULT_USER_PASS")
access_token = login(email=username, password=password)[0]

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /auth/login HTTP/1.1" 200 391


Use the access token from logging in to authenticate backend call. Remember, details of each endpoint can be found at http://localhost:5000/docs
 
The following command lists the existing users

In [4]:
auth_backend_call('get', '/user/list-users', token=access_token).json()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:http://localhost:5000 "GET /user/list-users HTTP/1.1" 200 26


['default_user@localhost']

Now lets add a user

In [5]:
payload = payload={
  "email": "user2@email.com",
  "password": "test_password"
}

auth_backend_call('post', '/user/create-user', payload=payload, token=access_token).json()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /user/create-user HTTP/1.1" 201 25


{'detail': 'User created'}

Now when we list the users, we see two users. 

In [6]:
auth_backend_call('get', '/user/list-users', token=access_token).json()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:http://localhost:5000 "GET /user/list-users HTTP/1.1" 200 44


['default_user@localhost', 'user2@email.com']